## Exercises II - Required

Let's set up an example scenario as perspective for our regression exercises using the Telco dataset.

As a customer analyst for Telco, you want to know who has spent the most money with the company over their lifetime. You have monthly charges and tenure, so you think you will be able to use those two attributes as features to estimate total charges. You need to do this within an average of $5.00 per customer. 

In these exercises, you will complete the first step toward the above goal: acquire and prepare the necessary Telco data from the telco_churn database in the Codeup database server.

1. Acquire `customer_id`, `monthly_charges`, `tenure`, and `total_charges` from the `telco_churn` database for all customers with a 2-year contract.
1. Using your acquired Telco data, walk through the summarization and cleaning steps in your `wrangle.ipynb` file like we did above. You may handle the missing values however you feel is appropriate and meaningful; remember to document your process and decisions using markdown and code commenting where helpful.
1. End with a `wrangle.py` file that contains the necessary functions to automate your process from acquiring the data to returning a cleaned dataframe with no missing values. Name your final function `wrangle_telco`.
---

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from env import host, username, password

Acquire Functions
Return customer_id, monthly_charges, tenure, and total_charges from telco_churn database for all customers with 2 year contracts.

Plan for creating my wrangle.py file.

- Include necessary imports to run my created functions in my wrangle.py file.
- Go to Sequel Pro first to create a SQL query that pulls what I need and make sure it works.
- Create a helper function to gain access to the database first providing me with the connection url I need. 
- Create a function that uses the query I created using Sequel Pro and the Pandas pd.read_sql(query, url) function to return a df to me.


In [3]:
# Create a helper function to provide connection url for Codeup database server.

def get_db_url(db_name):
    '''
    This function uses my env file to get the url to access the Codeup database.
    It takes in a string identifying the database I want to connect to.
    '''
    return f"mysql+pymysql://{username}:{password}@{host}/{db_name}"

In [4]:
# Create a generic function that takes in a database name and a query.

def get_data_from_sql(str_db_name, query):
    '''
    This function takes in a string for the name of the database I want to connect to
    and a query to obtain my data from the Codeup server and return a DataFrame.
    '''
    df = pd.read_sql(query, get_db_url(str_db_name))
    return df

In [5]:
query = """
        SELECT 
            customer_id, 
            monthly_charges, 
            tenure, 
            total_charges
        FROM customers
        WHERE contract_type_id = 3;
        """

df = get_data_from_sql('telco_churn', query)

In [6]:
df.head()

,customer_id,monthly_charges,tenure,total_charges
0,0013-SMEOE,109.70,71,7904.25
1,0014-BMAQU,84.65,63,5377.8
2,0016-QLJIS,90.45,65,5957.9
3,0017-DINOC,45.20,54,2460.55
4,0017-IUDMW,116.80,72,8456.75


In [7]:
print(f'My df has {df.shape[0]} rows and {df.shape[1]} columns.')

My df has 1695 rows and 4 columns.


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1695 entries, 0 to 1694
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      1695 non-null   object 
 1   monthly_charges  1695 non-null   float64
 2   tenure           1695 non-null   int64  
 3   total_charges    1695 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 53.1+ KB


In [10]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
monthly_charges,1695.0,60.770413,34.678865,18.4,24.025,64.35,90.45,118.75
tenure,1695.0,56.735103,18.209363,0.0,48.000,64.00,71.00,72.00


In [11]:
# customer_id is a unique identifyer for the customers
df.customer_id.nunique()

1695

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1695 entries, 0 to 1694
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      1695 non-null   object 
 1   monthly_charges  1695 non-null   float64
 2   tenure           1695 non-null   int64  
 3   total_charges    1695 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 53.1+ KB


Acquire and Summarize Takeaways
- total_charges is reading in as an object data type. This is unexpected. Could it contain a string like the student_grades dataset from the lesson?
- tenure has a minimum value of 0; do I want to keep them at 0, drop these observations, or what?
- I don't seem to have any Null values, but I might have some odd values to check out in total_charges.


# Prepare
- Walk through the steps from the lesson to clean the Telco DataFrame I acquired. Handle the missing values in an appropriate and meaningful way and document decisions.

In [13]:
df.head()

,customer_id,monthly_charges,tenure,total_charges
0,0013-SMEOE,109.70,71,7904.25
1,0014-BMAQU,84.65,63,5377.8
2,0016-QLJIS,90.45,65,5957.9
3,0017-DINOC,45.20,54,2460.55
4,0017-IUDMW,116.80,72,8456.75


In [15]:
# 'missing' values are causing the 'total_charges' and 'customer_id' columns to be an object type
# working on the 'total_charges' column below:
df.total_charges.value_counts(dropna=False)

           10
1110.05     2
5682.25     2
3533.6      2
343.45      2
           ..
804.85      1
5728.55     1
7616        1
6652.45     1
4060.9      1
Name: total_charges, Length: 1678, dtype: int64

- total_charges column has 10 values that are whitespaces. I can decide how to handle the 'missing' values and document my decision and actions taken.

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1695 entries, 0 to 1694
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      1695 non-null   object 
 1   monthly_charges  1695 non-null   float64
 2   tenure           1695 non-null   int64  
 3   total_charges    1695 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 53.1+ KB


-  I could simply drop these rows...

In [20]:
# Filter my dataframe to preserve only observations with total_charges; save to new df2.

df2 = df[df.total_charges != ' ']

In [21]:
# Validate that total_charges all have values.

df2[df2.total_charges == ' ']

,customer_id,monthly_charges,tenure,total_charges


In [22]:
# More validating...

df2.total_charges.value_counts(dropna=True).sort_index()

100.35     1
1004.35    1
1005.7     1
1006.9     1
1008.7     1
          ..
973.95     1
978        1
980.35     1
982.95     1
987.95     1
Name: total_charges, Length: 1677, dtype: int64

In [23]:
# I took care of the string values; now I can take care of the data type.

df2.total_charges.dtype

dtype('O')

In [24]:
df2.total_charges = df2.total_charges.astype(float)

In [25]:
# Validate my data types and non-null values. Now I have 1685 observsations.

df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1685 entries, 0 to 1694
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      1685 non-null   object 
 1   monthly_charges  1685 non-null   float64
 2   tenure           1685 non-null   int64  
 3   total_charges    1685 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 65.8+ KB


- BUT... It looks like those values are blank bc the tenure is 0. I will change it to 1 since they were probably customers for about a month.

In [26]:
df.tenure.value_counts().sort_index()

0      10
1       2
2       1
3       3
4       4
     ... 
68     65
69     66
70     88
71    137
72    343
Name: tenure, Length: 73, dtype: int64

In [27]:
# Replace any tenures of 0 with 1.

df.tenure = df.tenure.replace(0, 1)

In [28]:
# Validate my tenure count for value 1; I went from two to 12.

df.tenure.value_counts().sort_index()

1      12
2       1
3       3
4       4
5       1
     ... 
68     65
69     66
70     88
71    137
72    343
Name: tenure, Length: 72, dtype: int64

In [29]:
# These observations also need total_charges handled. 
# The other tenure 1 observations have same monthly and total charges.

df[df.tenure == 1]

,customer_id,monthly_charges,tenure,total_charges
188,1099-GODLO,20.35,1,20.35
234,1371-DWPAZ,56.05,1,
416,2520-SGTTA,20.00,1,
453,2775-SEFEE,61.90,1,
505,3115-CZMZD,20.25,1,
524,3213-VVOLG,25.35,1,
678,4075-WKNIU,73.35,1,
716,4367-NUYAO,25.75,1,
726,4472-LVYGI,52.55,1,
941,5709-LVOEQ,80.85,1,


In [30]:
# Replace the blank total_charges with the monthly_charge for tenure == 1.

df.total_charges = np.where(df.total_charges==' ', df.monthly_charges, df.total_charges)

In [31]:
# Validate my changes.

df[df.tenure == 1]

,customer_id,monthly_charges,tenure,total_charges
188,1099-GODLO,20.35,1,20.35
234,1371-DWPAZ,56.05,1,56.05
416,2520-SGTTA,20.00,1,20
453,2775-SEFEE,61.90,1,61.9
505,3115-CZMZD,20.25,1,20.25
524,3213-VVOLG,25.35,1,25.35
678,4075-WKNIU,73.35,1,73.35
716,4367-NUYAO,25.75,1,25.75
726,4472-LVYGI,52.55,1,52.55
941,5709-LVOEQ,80.85,1,80.85


In [32]:
df.total_charges.dtype

dtype('O')

In [ ]:
# Handle the data type for total_charges.

df.total_charges = df.total_charges.astype(float)

In [33]:
# Validate my data type conversion.

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1695 entries, 0 to 1694
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      1695 non-null   object 
 1   monthly_charges  1695 non-null   float64
 2   tenure           1695 non-null   int64  
 3   total_charges    1695 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 53.1+ KB


# Prepare Function
- End with a python file wrangle.py that contains the function, wrangle_telco(), that will acquire the data and return a dataframe cleaned with no missing values.
- I will create my wrangle_telco() function and a wrangle.py file to reflect the prep I want.
- Again, all the hard work I did above will be the guts in my wrangle_telco function making it very simple to acquire and prepare this data in the same way, making it a repeatable process.
- Remember, this is the basic process you will go through in projects to aquire and prep your data and create modules containing the functions you build along the way.
- As a bonus, I'll create a reusable splitting function.

In [34]:
def wrangle_telco():
    """
    Queries the telco_churn database
    Returns a clean df with four columns:
    customer_id(object), monthly_charges(float), tenure(int), total_charges(float)
    """
    query = """
            SELECT 
                customer_id, 
                monthly_charges, 
                tenure, 
                total_charges
            FROM customers
            WHERE contract_type_id = 3;
            """
    df = get_data_from_sql('telco_churn', query)
    
    # Replace any tenures of 0 with 1
    df.tenure = df.tenure.replace(0, 1)
    
    # Replace the blank total_charges with the monthly_charge for tenure == 1
    df.total_charges = np.where(df.total_charges==' ', df.monthly_charges, df.total_charges) 
    
    # Convert total_charges to a float.
    df.total_charges = df.total_charges.astype(float)
    
    return df

In [35]:
# Validate I can call my function and return what I expect.

telco = wrangle_telco()
telco.head()

,customer_id,monthly_charges,tenure,total_charges
0,0013-SMEOE,109.70,71,7904.25
1,0014-BMAQU,84.65,63,5377.80
2,0016-QLJIS,90.45,65,5957.90
3,0017-DINOC,45.20,54,2460.55
4,0017-IUDMW,116.80,72,8456.75


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1695 entries, 0 to 1694
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      1695 non-null   object 
 1   monthly_charges  1695 non-null   float64
 2   tenure           1695 non-null   int64  
 3   total_charges    1695 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 53.1+ KB


In [ ]:
# Test the function: 

In [37]:
# Splitting function

In [38]:
def split_continuous(df):
    '''
    Takes in a df
    Returns train, validate, and test DataFrames
    '''
    # Create train_validate and test datasets
    train_validate, test = train_test_split(df, 
                                        test_size=.2, 
                                        random_state=123)
    # Create train and validate datsets
    train, validate = train_test_split(train_validate, 
                                   test_size=.3, 
                                   random_state=123)

    # Take a look at your split datasets

    print(f'train -> {train.shape}')
    print(f'validate -> {validate.shape}')
    print(f'test -> {test.shape}')
    return train, validate, test

In [39]:
train, validate, test = split_continuous(df)

train -> (949, 4)
validate -> (407, 4)
test -> (339, 4)


In [40]:
train.head()

,customer_id,monthly_charges,tenure,total_charges
1256,7501-IWUNG,73.80,61,4616.05
225,1303-SRDOK,69.05,55,3842.6
662,3967-VQOGC,24.90,67,1680.25
628,3777-XROBG,19.55,58,1079.65
824,5075-JSDKI,24.45,59,1493.1


In [41]:
train.head()

,customer_id,monthly_charges,tenure,total_charges
1256,7501-IWUNG,73.80,61,4616.05
225,1303-SRDOK,69.05,55,3842.6
662,3967-VQOGC,24.90,67,1680.25
628,3777-XROBG,19.55,58,1079.65
824,5075-JSDKI,24.45,59,1493.1
